In [1]:
import os 
import tensorflow as tf

In [2]:
root_dir = "../"
root_dir = os.path.abspath(root_dir)
data_path = os.path.join(root_dir, "data")
processed_path = os.path.join(data_path, "processed")

In [161]:
file_list =[os.path.join(processed_path, x) for x in os.listdir(processed_path)]
file_list = file_list[1:]#.gitkeep
file_list

['c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record1.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record10.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record11.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record12.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record13.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record14.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record15.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record16.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record17.tfrecords',
 'c:\\Users\\willi\\Desktop\\AIPortfolio\\medical_imaging\\data\\processed\\record18.tfrecords',
 'c:\\Users\\willi\\Desktop\\AI

In [162]:
def parse_tfrecord_fn(example_proto):
    feature_description = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }

    example = tf.io.parse_single_example(example_proto, feature_description)
    
    height = tf.cast(example['height'], tf.int32)
    width = tf.cast(example['width'], tf.int32)
    depth = tf.cast(example['depth'], tf.int32)
    label = tf.cast(example['label'], tf.int32)

    image_raw = tf.io.decode_raw(example['image_raw'], tf.int32)
    image_raw = tf.cast(image_raw, tf.float32)


    image = tf.reshape(image_raw, (256, 256, 38))
    label = tf.reshape(label, (1,))



     
    return image, label

In [163]:
tf_set = tf.data.TFRecordDataset(file_list)
tf_set

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [164]:
tf_set = tf_set.map(parse_tfrecord_fn)
tf_set

<_MapDataset element_spec=(TensorSpec(shape=(256, 256, 38), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.int32, name=None))>

In [165]:
# Create an iterator and get the next element
iterator = iter(tf_set)
for i in range(1):
    image, label = iterator.get_next()
    print(image.shape)

(256, 256, 38)


In [166]:
tf_set=tf_set.shuffle(1000).batch(10, drop_remainder=True)
tf_set

<_BatchDataset element_spec=(TensorSpec(shape=(10, 256, 256, 38), dtype=tf.float32, name=None), TensorSpec(shape=(10, 1), dtype=tf.int32, name=None))>

In [167]:
import keras

class DenseOutput(keras.layers.Layer):
    def __init__(self, output_classes:int):
        super().__init__()
        self.flatten_layer = keras.layers.Flatten()
        self.dense_1 = keras.layers.Dense(4096, activation='relu')
        self.dense_2 = keras.layers.Dense(4096, activation='relu')
        self.output_layer = keras.layers.Dense(output_classes, activation='softmax')

    def call(self, x):
        x = self.flatten_layer(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.output_layer(x)
        return x

In [168]:
class ConvComponent(keras.layers.Layer):
    def __init__(self, channel_count: int):
        super().__init__()
        self.conv_1 = keras.layers.Conv2D(channel_count, (3, 3), activation='relu', padding='same')
        self.conv_2 = keras.layers.Conv2D(channel_count, (3, 3), activation='relu', padding='same')
        self.pool_layer = keras.layers.MaxPooling2D((2,2), strides=(2,2))

    def call(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.pool_layer(x)
        return x


In [169]:
class Vgg16(keras.Model):
    def __init__(self, output_classes:int):
        super().__init__()
        self.initial_pool = keras.layers.MaxPooling2D((5, 5), strides=(5, 5))
        self.comp_1 = ConvComponent(64)
        self.comp_2 = ConvComponent(128)
        self.comp_3 = ConvComponent(256) #all these have only 2 components (slight deviation)
        self.comp_4 = ConvComponent(512)
        self.comp_5 = ConvComponent(512)
        self.connected_layer = DenseOutput(output_classes)

    def call(self, inputs):
        inputs = self.initial_pool(inputs)
        inputs = self.comp_1(inputs)
        inputs = self.comp_2(inputs)
        inputs = self.comp_3(inputs)
        inputs = self.comp_4(inputs)
        inputs = self.comp_5(inputs)
        inputs = self.connected_layer(inputs)
        return inputs

In [172]:
A = Vgg16(4)
A.compile("adam", keras.losses.SparseCategoricalCrossentropy())
A.fit(tf_set, epochs=10)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 569ms/step - loss: 1846.1052
Epoch 2/10


c:\Users\willi\Anaconda3\envs\Medical_Env\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 347ms/step - loss: 0.2255
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 343ms/step - loss: 1.2864  
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 334ms/step - loss: 0.0963
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 348ms/step - loss: 0.2460
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 352ms/step - loss: 0.1705
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 356ms/step - loss: 0.2261
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 317ms/step - loss: 0.0430
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 330ms/step - loss: 0.6215
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 319ms/step - loss: 0.2025


In [ ]:
for batch in tf_set:
    print(f"1: {batch[0].shape}")
    print(f"2: {batch[1].shape}")
    #print(f"HEREL {batch[0]}")
    #brak

In [ ]:
tf_set = tf.data.TFRecordDataset(file_list)
tf_set = tf_set.map(parse_tfrecord_fn)
#tf.convert_to_tensor(tf_set)
#tf_set = tf_set.map(test_func)
#tf_set = tf_set.map(lambda x, y: tf.ensure_shape(x, (256, 256, 38)))
#tf_set = tf_set.map(lambda x, y: tf.cast(x, (256, 256, 38)))
#tf_set=tf_set.shuffle(1000).batch(10, drop_remainder=True)
#tf_set